In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chandramaniyadav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('item_feature_python_extract_Training_all.csv', encoding = 'utf8')

/var/folders/y_/93fvbts118bgkwysz7wjp5p80000gn/T/ipykernel_40337/2496371385.py:1: DtypeWarning: Columns (10,12,15,19,20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('item_feature_python_extract_Training_all.csv', encoding = 'utf8')


In [4]:
# List of unique accessions
unique_accessions = ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']



# Dictionary to store the dataframes
dfs = {}

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the dataframe name
    
    df_name = 'df_' + accession

    # Read the CSV file into a dataframe
    df = pd.read_csv( df_name + '.csv')

    # Add the dataframe to the dictionary
    dfs[accession] = df

/var/folders/y_/93fvbts118bgkwysz7wjp5p80000gn/T/ipykernel_40337/3100533466.py:17: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv( df_name + '.csv')


In [5]:
df = dfs['VH304954']

In [6]:
df['predict_from']

0        Mark needs to do next is to add 5 to the answe...
1                             he needs to take away 43 -48
2         What he needs to do next is subtract 5 from 100.
3                          he needs to subtract 5 from 100
4                            he needs to solve the problem
                               ...                        
19550                                 subtract 45 from 100
19551    Subtract 95 from 143-43 which the answer for 1...
19552                     he needs to subtract 100 from 48
19553                                    the answer is 405
19554                   know he need's to subtract  143-48
Name: predict_from, Length: 19555, dtype: object

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
rest_texts, test_texts, rest_labels, test_labels = train_test_split(df['predict_from'], df['assigned_score'], test_size=0.1, random_state=1)
train_texts, dev_texts, train_labels, dev_labels = train_test_split(rest_texts, rest_labels, test_size=0.1, random_state=1)


In [9]:
print("Train size:", len(train_texts))
print("Dev size:", len(dev_texts))
print("Test size:", len(test_texts))

Train size: 15839
Dev size: 1760
Test size: 1956


In [10]:
train_labels = train_labels-1
dev_labels = dev_labels-1
test_labels = test_labels-1

In [11]:
!pip install transformers==4.28.0

In [12]:
MODEL = "distilbert-base-uncased"

In [13]:
!pip install datasets

In [14]:
from datasets import Dataset,load_dataset, load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, AdamW

# Create model and tokenizer
# Make sure the num_labels argument matches the question (it will usually be 2, for correct/incorrect)
# Some questions may require more than one model (for more than one written section)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

/Users/chandramaniyadav/miniconda3/envs/torch-gpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertFor

In [15]:
# Some questions may require more than one model (for more than one written section)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [16]:
df['labels'] = df['score_to_predict'] - 1

In [17]:
# Convert to dataset format
dataset = Dataset.from_pandas(df, preserve_index=False)
dataset = dataset.train_test_split(test_size=0.2, seed=11)
dataset

DatasetDict({
    train: Dataset({
        features: ['student_id', 'accession', 'score_to_predict', 'predict_from', 'year', 'srace10', 'dsex', 'accom2', 'iep', 'lep', 'rater_1', 'pta_rtr1', 'ptb_rtr1', 'ptc_rtr1', 'composite', 'score', 'assigned_score', 'ee_use', 'parsed_xml_v1', 'parsed_xml_v2', 'parsed_xml_v3', 'source1', 'source2', 'source3', 'source4', 'target1', 'target2', 'target3', 'target4', 'eliminations', 'selected', 'eliminated', 'selected1', 'selected2', 'selected3', 'selected4', 'eliminated1', 'eliminated2', 'eliminated3', 'eliminated4', 'selected1.1', 'selected2.1', 'eliminated1.1', 'eliminated2.1', 'partA_response_val', 'partB_response_val', 'partB_eliminations', 'predict_from_withstop', 'predict_from_onestepall', 'n_letter_all', 'n_sentence', 'n_token_verb', 'n_token_adj', 'n_token_noun', 'n_token_adv', 'n_token_content', 'n_token_content_unique', 'n_token_all', 'n_token_alpha', 'n_token_unique', 'word_diversity', 'mean_word_length', 'n_token_non_DaleChall', 'n_token_f

In [18]:
# Tokenize the data
model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    return tokenizer(examples["parsed_xml_v1"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [19]:
pip install --upgrade accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [20]:
# AdamW Training
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=None,
)
trainer.train()

/Users/chandramaniyadav/miniconda3/envs/torch-gpu/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 2/1467 [00:43<8:53:40, 21.86s/it]

KeyboardInterrupt: 